# Road Lane Detection
## (Self-Guided Project)

## by Justin Sierchio

In this project, we will be looking at road lane detections.

This data is in .csv file format and is from DataFlair at: https://data-flair.training/blogs/road-lane-line-detection/. More information related to the dataset can be found at the same link.

Note: this is a self-guided project following the tutorial provided the contributors at DataFlair.

## Notebook Initialization

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
import math

print('Initial libraries loaded into workspace!')

Initial libraries loaded into workspace!


## Description

Specifically, we are going to look at the road lanes where autonomous cars will run. We will be trying to detect white markings for the lanes by masking the rest of the frame. Each frame is a NumPy array of values and we will be converting those to 0. Once the lanes are detected, we will be using a Hough Transform (which detects rectangles, circles, triangles and lines. 